In [ ]:
#@title Imports.
import sympy as sp
from tqdm import tqdm
import functools
import itertools
import math

# Theory

Let us fix a finite field $GF(q)$ and some positive integers $m, n$. Let us denote the set of  $m \times n$ matrices of rank $k$ as $\mathcal{J}_k$.

Consider a fixed matrix $A$ of rank $i$. Then consider the intersection $\mathcal{J}_j \cap (\mathcal{J}_k + A)$. The number of elements in this intersection we denote $d_{ijk}(m, n)$. It does not depend on choice of $A$.

We obtained a theoretic formula for $d_{ijk}(m,n)$:

$$
\newcommand{\qbinom}[2]{\begin{bmatrix}#1 \\ #2 \end{bmatrix}_q}
\newcommand{\factq}[1]{\langle #1 \rangle!}
d_{ijk}(m,n) = \sum_{r = n - j}^n (-1)^{r-n+j} \cdot q^{\binom{r-n+j}{2}}\cdot \qbinom{r}{n-j} \
    \sum\limits_{x=\max(0, r-i)}^{\min (n-i,r)} \qbinom{n-i}{x} \qbinom{i}{r-x} \times \\ \times q^{(r-x)(n-i-x)} \sum\limits_{z=\max(0, k+x-r)}^{\min (k,n-r)} \qbinom{r-x}{z+r-x-k} \cdot
 \qbinom{m+x-r}{k+x-r} q^{(k+x-r)(k-z)} q^{\binom{z}{2}} \frac{\factq{n-r}}{\factq{n-r - z}}.
$$

Let us consider those numbers as a sequence in $i$ for any $q,m,n$ and $j=k$. Our current goal is to prove that such sequence is strictly descending.

# Preparation

In this section we prepare several necessary functions that will perform the checks.

Function `d_mnijk_poly` accepts a symbol `q` and numbers $m,n,i,j,k$. It produces the number $d_{ijk}(m,n)$ as a polynomial in $q$.

In [ ]:
@functools.cache
def qfact_as_poly (q, n):
  prod = 1
  for i in range(1, n+1):
    prod *= (q**i-1) # the sign means that we get not (q,q)_n but |q,q|_n
  return prod

@functools.cache
def q_fraction_as_poly (q, n, k):
  if (n<0) or (k<0) or (k>n):
    return 0
  ans = 1
  for i in range(k+1, n+1):
    ans *= (q**i-1)
  return ans

@functools.cache
def q_binomial_as_poly (q, n, k):
    if k < 0 or k > n:
        return 0
    if k == 0 or k == n:
        return 1
    return q_binomial_as_poly(q, n - 1, k - 1) + q ** k * q_binomial_as_poly(q, n - 1, k)

def d_mnijk_poly (q, m, n, i, j, k):
  ans = 0
  try:
    for r in range(n-k, n+1):
      sum = 0
      for x in range(max(0,r-i,r-j), min(n-i,r)+1):
        sum2 = 0
        for z in range(j+x-r, min(j,n-r)+1):
          sum2 += q_binomial_as_poly(q, r-x, z+r-x-j)*(q**((j+x-r)*(j-z)+int((z)*(z-1)//2)))*(q_fraction_as_poly(q,n-r,n-r-z))
        sum += sum2 * q_binomial_as_poly(q, n-i, x) * q_binomial_as_poly(q, m+x-r, j+x-r) * q_binomial_as_poly(q, i, r-x) * (q**((r-x)*(n-i-x)))
      ans += sum * ((-1)**(r-n+k)) * (q**(int((r-n+k)*(r-n+k-1)//2))) * q_binomial_as_poly(q, r, n-k)
  except:
    print(f'Error: m={m}, n={n}, i={i}, j={j}, k={k}, r={r}, x={x}, z={z}')
    return
  return ans

q = sp.Poly(sp.symbols('q'))

We rewrite our formula also in the form
$$
d_{ijk}(m,n)=\sum_{s=0}^{k} q^{\binom{s}{2} + s} (-1)^s q^{(m-k)s} \sum_{r = n - j}^n (-1)^{r-n+j} \cdot q^{\binom{r-n+j}{2}}\cdot \qbinom{r}{n-j} \
\sum\limits_{x=\max(0, r-i, r - k, r+s-k)}^{\min (n-i,r)} \qbinom{n-i}{x} \qbinom{i}{r-x} \times
$$
$$
q^{(r-x)(n-i-x)} \sum\limits_{z=k+x-r}^{\min (k,n-r)} \qbinom{r-x}{z+r-x-k} \cdot
$$
$$
\qbinom{n-r}{n-r-z} q^{(k+x-r)(k-z)} q^{\binom{z}{2}} \frac{\factq{z}}{\factq{k+x-r}} (-1)^{k+x-r} \qbinom{k+x-r}{s} =
$$
$$
= \sum_{s=0}^{k} q^{(m-k)s} G_s(q,n,i,j,k).
$$

We rewrite the function so that it finds those polynomials $G_s(q,n,i,j,k)$ separately.

In [ ]:
def d_mnijk_poly_clusters (q, n, i, j, k):
  ans = []
  try:
    for s in range(0,j+1):
      sum0 = 0
      for r in range(n-k, n+1):
        sum = 0
        for x in range(max(0,r-i,r-j+s), min(n-i,r)+1):
          sum2 = 0
          for z in range(j+x-r, min(j,n-r)+1):
            sum2 += q_binomial_as_poly(q, r-x, z+r-x-j)*(q**((j+x-r)*(j-z)+int((z)*(z-1)//2)))*q_binomial_as_poly(q, n-r, n-z-r) *q_binomial_as_poly(q, j+x-r, s) *(q_fraction_as_poly(q,z,j+x-r))*((-1)**(j+x-r))
          sum += sum2 * q_binomial_as_poly(q, n-i, x) * q_binomial_as_poly(q, i, r-x) * (q**((r-x)*(n-i-x)))
        sum0 += sum * ((-1)**(r-n+k)) * (q**(int((r-n+k)*(r-n+k-1)//2))) * q_binomial_as_poly(q, r, n-k)
      ans.append(sum0*((-1)**s)*q**(int(s*(s-1)/2) + s)) # we need to further multiply by q**(s*(m-j))
  except:
    print(f'Error: n={n}, i={i}, j={j}, k={k}, r={r}, x={x}, z={z}')
    return
  return ans

def generate_for_all_i_clusters (q, n, j, k):
  ans = []
  for i in range(0,n+1):
    ans.append(d_mnijk_poly_clusters(q,n,i,j,k))
  return ans

def differences_for_all_i_clusters (q, n, j, k, draft=False):
  ans = []
  if draft:
    print(f'n={n}, j={j}, k={k}')
  clusters = generate_for_all_i_clusters(q,n,j,k)
  for i in range(len(clusters)-1):
    if draft:
      print(f'i={i}')
    ans.append([clusters[i][s]-clusters[i+1][s] for s in range(len(clusters[i]))])
    if draft:
      for s in range(len(clusters[i])):
        print(clusters[i][s]-clusters[i+1][s])
  return ans

Function `check_poly_positivity_fast` checks a given polynomial for being positive when $q\geq 2$ by substuting $q+2$ instead of $q$, expanding and checking that all coefficients are positive. This is not equivalent to positivity (for example, $q^2-q+1$ is positive for all $q \geq 0$ despite one of the coefficients being negative), but it is a sufficient condition. We will see later that it holds for most cases that are relevant to us.

The function accepts the polynomial itself, the `sympy` variable in which the polynomial is given, and an optional argument `shift` that allows to switch from shifting by 2 to any other value.

In [ ]:
def check_poly_positivity_fast(polynomial, variable, shift):
  #polynomial_shifted = sp.Poly(polynomial.subs(variable, variable+shift))
  polynomial_shifted_coeffs = (sp.compose(polynomial, variable+shift)).all_coeffs()
  return (polynomial_shifted_coeffs[-1] > 0) and all([c >= 0 for c in polynomial_shifted_coeffs])

In [ ]:
def check_differences_for_all_i (q, n, j, k, margin=2):
  differences = differences_for_all_i_clusters(q,n,j,k)
  for i in range(len(differences)):
    seeking_largest = True
    non_leading_coeffs_sum = 0
    non_leading_power = 0
    first_suspicious_s = 0

    for s in reversed(range(len(differences[i]))):
      if seeking_largest and (differences[i][s] == 0):
        continue
      elif seeking_largest:
        seeking_largest = False
        leading_part = differences[i][s]
        leading_coeffs = leading_part.all_coeffs()
        problem = False
        for t in leading_coeffs:
          if t < 0:
            print(f'n={n},i={i},j={j},k={k}')
            print(differences[i])
            problem = True
        if problem:
          continue
        the_leading_coeff = leading_coeffs[0]
        the_leading_power = leading_part.degree()
        the_largest_s = s
      elif not check_poly_positivity_fast(polynomial=differences[i][s],variable=q, shift=margin):
        non_leading_coeffs_sum += sp.Add(*[abs(coeff) for coeff in differences[i][s].all_coeffs()])
        non_leading_power = max(non_leading_power, differences[i][s].degree())
        first_suspicious_s = max(first_suspicious_s, s)
    # we try to make
    # q^{m-k}*(the_largest_s-first_suspicious_s) * q^the_leading_power * the_leading_coeff
    # >
    # non_leading_coeffs_sum * q^non_leading_power
    # for all q>=2
    # so we want m-k > log_q (non_leading_coeffs_sum/leading_coeffs_sum)
    # we pick the smallest q we consider -- the parameter `margin`
    if non_leading_coeffs_sum == 0:
      min_m = 0
    else:
      min_m = math.ceil(((math.log(non_leading_coeffs_sum/the_leading_coeff,margin))+non_leading_power-the_leading_power)/(the_largest_s-first_suspicious_s))+k
    for m in range(n, min_m+1):
      cur = sp.cancel((d_mnijk_poly(q,m,n,i,j,k)-d_mnijk_poly(q,m,n,i+1,j,k)))
      if not check_poly_positivity_fast(cur, q, margin):
        print(f'\nm={m},n={n},i={i},j={j},k={k},ans={cur}')

In [ ]:
for n in tqdm(range(2,14)):
  for j in itertools.chain(range(1, n-1), range(n, n+1)): # we skip j=m-1
    k = j
    check_differences_for_all_i(q=q,n=n,j=j,k=k)

 17%|█▋        | 2/12 [00:00<00:00, 15.85it/s]


m=2,n=2,i=1,j=2,k=2,ans=Poly(q**2 - 2*q, q, domain='ZZ')


100%|██████████| 12/12 [06:20<00:00, 31.72s/it]


In [ ]:
for n in tqdm(range(2,14)):
  for j in range(n-1, n): # case j=m-1
    k = j
    check_differences_for_all_i(q=q,n=n,j=j,k=k)

  0%|          | 0/12 [00:00<?, ?it/s]


m=2,n=2,i=1,j=1,k=1,ans=Poly(q**2 - 2*q - 2, q, domain='ZZ')

m=3,n=3,i=1,j=2,k=2,ans=Poly(q**7 - 2*q**6 - 2*q**5 + 3*q**4 + 2*q**3 - q**2 - 2*q, q, domain='ZZ')

m=3,n=3,i=2,j=2,k=2,ans=Poly(q**6 - 2*q**5 - 3*q**4 + 3*q**3 + 5*q**2 + 3*q, q, domain='ZZ')


 25%|██▌       | 3/12 [00:00<00:00,  9.96it/s]


m=4,n=4,i=3,j=3,k=3,ans=Poly(q**12 - 2*q**11 - 3*q**10 + 2*q**9 + 7*q**8 + 7*q**7 - 4*q**6 - 9*q**5 - 8*q**4 - 4*q**3, q, domain='ZZ')


 33%|███▎      | 4/12 [00:00<00:01,  4.57it/s]

...

 83%|████████▎ | 10/12 [00:49<00:22, 11.00s/it]


m=12,n=12,i=11,j=11,k=11,ans=Poly(q**132 - 2*q**131 - 3*q**130 + 2*q**129 + 6*q**128 + 8*q**127 + q**126 - 6*q**125 - 14*q**124 - 18*q**123 - 12*q**122 - 2*q**121 + 11*q**120 + 23*q**119 + 30*q**118 + 29*q**117 + 31*q**116 + 27*q**115 + 10*q**114 - 21*q**113 - 58*q**112 - 90*q**111 - 106*q**110 - 103*q**109 - 75*q**108 - 24*q**107 + 47*q**106 + 117*q**105 + 174*q**104 + 220*q**103 + 233*q**102 + 205*q**101 + 125*q**100 + 13*q**99 - 122*q**98 - 254*q**97 - 355*q**96 - 410*q**95 - 404*q**94 - 328*q**93 - 203*q**92 - 29*q**91 + 170*q**90 + 356*q**89 + 498*q**88 + 566*q**87 + 555*q**86 + 457*q**85 + 300*q**84 + 97*q**83 - 116*q**82 - 315*q**81 - 472*q**80 - 570*q**79 - 582*q**78 - 503*q**77 - 360*q**76 - 186*q**75 - 7*q**74 + 152*q**73 + 278*q**72 + 359*q**71 + 391*q**70 + 375*q**69 + 316*q**68 + 223*q**67 + 98*q**66 - 2*q**65 - 72*q**64 - 115*q**63 - 136*q**62 - 139*q**61 - 129*q**60 - 109*q**59 - 84*q**58 - 57*q**57 - 32*q**56 - 12*q**55, q, domain='ZZ')


 92%|█████████▏| 11/12 [01:25<00:18, 18.43s/it]


m=13,n=13,i=12,j=12,k=12,ans=Poly(q**156 - 2*q**155 - 3*q**154 + 2*q**153 + 6*q**152 + 8*q**151 + q**150 - 6*q**149 - 14*q**148 - 18*q**147 - 12*q**146 - 2*q**145 + 10*q**144 + 25*q**143 + 34*q**142 + 27*q**141 + 23*q**140 + 17*q**139 + 8*q**138 - 13*q**137 - 41*q**136 - 68*q**135 - 91*q**134 - 101*q**133 - 88*q**132 - 51*q**131 + 11*q**130 + 83*q**129 + 141*q**128 + 190*q**127 + 221*q**126 + 226*q**125 + 188*q**124 + 112*q**123 - 4*q**122 - 137*q**121 - 267*q**120 - 378*q**119 - 451*q**118 - 455*q**117 - 394*q**116 - 269*q**115 - 88*q**114 + 125*q**113 + 350*q**112 + 540*q**111 + 676*q**110 + 726*q**109 + 683*q**108 + 545*q**107 + 333*q**106 + 59*q**105 - 232*q**104 - 516*q**103 - 745*q**102 - 874*q**101 - 893*q**100 - 805*q**99 - 623*q**98 - 369*q**97 - 78*q**96 + 220*q**95 + 486*q**94 + 692*q**93 + 816*q**92 + 842*q**91 + 748*q**90 + 575*q**89 + 355*q**88 + 125*q**87 - 93*q**86 - 276*q**85 - 415*q**84 - 496*q**83 - 521*q**82 - 489*q**81 - 410*q**80 - 294*q**79 - 144*q**78 - 20*q**7

100%|██████████| 12/12 [02:21<00:00, 11.81s/it]


We see that the exceptional cases where this approach does not work are $i=j=k=m-1=n-1$ and additionally one special case $m=n=3, i=1, j=k=2$. We are going to check for them that for $q\geq 3$ the approach works:

In [ ]:
for n in tqdm(range(2,14)):
  for j in range(n-1, n): # case j=n-1
    k = j
    check_differences_for_all_i(q=q,n=n,j=j,k=k, margin=3)

100%|██████████| 12/12 [01:26<00:00,  7.19s/it]


Let us print the values $d_{ijk}(m,n)$ in the remaining cases:

In [ ]:
for n in range(2,14):
  m = n
  for j in range(n-1, n): # case j=n-1
    k = j
    print(f'm=n={n}, j=k={k}')
    for i in range(abs(j-k),min(m,n,j+k)+1):
      print(f'i={i}', d_mnijk_poly(2,m,n,i,j,k)) # here we abuse that if we replace q by number the function continues to work properly

m=n=2, j=k=1
i=0 9
i=1 4
i=2 6
m=n=3, j=k=2
i=0 294
i=1 162
i=2 170
i=3 168
m=n=4, j=k=3
i=0 37800
i=1 22344
i=2 21816
i=3 21776
i=4 21840
m=n=5, j=k=4
i=0 19373760
i=1 11793600
i=2 11250624
i=3 11184192
i=4 11184064
i=5 11189760
m=n=6, j=k=5
i=0 39687459840
i=1 24488432640
i=2 23144002560
i=3 22946939904
i=4 22919740416
i=5 22919643136
i=6 22922532864
m=n=7, j=k=6
i=0 325139829719040
i=1 201929795665920
i=2 190047036211200
i=3 188170346004480
i=4 187837850812416
i=5 187787294834688
i=6 187787202560000
i=7 187793024090112
m=n=8, j=k=7
i=0 10654345790226432000
i=1 6638054763543920640
i=2 6235071511642767360
i=3 6169220238393999360
i=4 6156520972458393600
i=5 6154043897860325376
i=6 6153649504967983104
i=7 6153649132377473024
i=8 6153696450308997120
m=n=9, j=k=8
i=0 1396491759480328106803200
i=1 871440250874200326144000
i=2 817748542931991779082240
i=3 808830536200640759070720
i=4 807047479852516025303040
i=5 806668880090392166400000
i=6 806591463308316666494976
i=7 806578851648469003665

Is it not very hard to see by hand that while the sequences are not monotoneous anymore, all numbers in each sequence are distinct. To be sure, we ask the computer to check this for us:

In [ ]:
for n in tqdm(range(2,14)):
  m = n
  for j in range(n-1, n): # case j=n-1
    k = j
    seq = []
    for i in range(abs(j-k),min(m,n,j+k)+1):
      seq.append(d_mnijk_poly(2,m,n,i,j,k)) # here we abuse that if we replace q by number the function continues to work properly
    if len(set(seq)) != len(seq):
      print(f'm=n={n}, j=k={k}')
      print(seq)

100%|██████████| 12/12 [00:00<00:00, 1589.81it/s]
